In [2]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image
from scipy import sparse
import cv2
from pymatreader import read_mat
# from extract_graph import dic_to_sparse
from util import get_path, shift_skeleton
from plotutil import show_im,overlap, show_im_rgb, plot_nodes, plot_nodes_from_list,plot_t_tp1
from extract_graph import generate_graph_tab_from_skeleton,generate_nx_graph_from_skeleton,generate_skeleton,clean
import networkx as nx
from node_id import second_identification, whole_movement_identification,first_identification,relabel_nodes, clean_nodes, orient
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab
from sparse_util import dilate, zhangSuen
from realign import realign, reconnect
from util import get_path
import pandas as pd

In [3]:
# date0='0709_1157'
# date1='0709_1557'
# date2='0709_1934'
date0='0701_1957'
date1='0701_2357'
date2='0702_0357'
date3='0702_0757'
date4='0702_1157'
date5='0702_1557'
plate=13
dates=[date0,date1,date2,date3,date4,date5]


In [3]:
mat_skels=[read_mat(get_path(date,plate,True))['skel'] for date in dates]

In [4]:
dic_skels=[dic_to_sparse(mat_skel) for mat_skel in mat_skels]

In [ ]:
sub_skeleton_docs_aligned = [realign(skeleton_doc,skeleton_docs[0]) for skeleton_doc in skeleton_docs]

In [8]:
skeleton_docs_aligned = [reconnect(skeleton_docs[0])]+sub_skeleton_docs_aligned 

872558 872271
107117 796
394 60
35 28
17 15
10 0
0 0


In [10]:
for i, date in enumerate([date0,date1,date2,date3,date4,date5]):
    sparse.save_npz(f'Data/skeleton_{date}_{plate}_full',sparse.csc_matrix(skeleton_docs_aligned[i]))

In [4]:
skeleton_docs_aligned = [sparse.dok_matrix(sparse.load_npz(f'Data/skeleton_{date}_{plate}_full.npz')) for date in dates]

In [4]:
from plotutil import plot_t_tp1, compress_skeleton
factor = 5
final_pictures = [compress_skeleton(skeleton_docs_aligned[i],factor) for i in range(len(skeleton_docs_aligned))]

In [36]:
plot_t_tp1([],[],poss_aligned[4],poss_aligned[5],final_pictures[3],final_pictures[4],compress=5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
graph_tabs=[from_sparse_to_graph(skeleton) for skeleton in skeleton_docs_aligned]

In [6]:
nx_graph_poss=[generate_nx_graph(graph_tab) for graph_tab in graph_tabs]
nx_graphs_aligned=[nx_graph_pos[0] for nx_graph_pos in nx_graph_poss]
poss_aligned=[nx_graph_pos[1] for nx_graph_pos in nx_graph_poss]
nx_graph_pruned=[prune_graph(nx_graph) for nx_graph in nx_graphs_aligned]

In [7]:
def sub_graph(nx_graph,pos,xbegin,xend,ybegin,yend):
    sub_nx_graph=nx.Graph.copy(nx_graph)
    for node in nx_graph.nodes:
        if (pos[node][0]>=xend or pos[node][0]<=xbegin or pos[node][1]>=yend or pos[node][1]<=ybegin):
            sub_nx_graph.remove_node(node)
    return(sub_nx_graph)

In [8]:
def generate_skeleton(nx_graph,dim,shift=(0,0)):
    skel = sparse.dok_matrix(dim, dtype=bool)
    for edge in nx_graph.edges.data('pixel_list'):
        for pixel in edge[2]:
            skel[(pixel[0]-shift[0],pixel[1]-shift[1])]=True
    return(skel)

In [14]:
poss_aligned=[nx_graph_pos[1] for nx_graph_pos in nx_graph_poss]
nx_graph_pruned=[prune_graph(nx_graph) for nx_graph in nx_graphs_aligned]

In [10]:
xbegin=14000
xend = 18000
ybegin = 8000
yend=16000
sub_nx_graphs = [sub_graph(nx_graph,poss_aligned[i],xbegin,xend,ybegin,yend) for i, nx_graph in enumerate(nx_graph_pruned)]
# sub_skeleton= [generate_skeleton(nx_graph,(xend-xbegin,yend-ybegin),(xbegin,ybegin)) for nx_graph in sub_nx_graphs]

In [13]:
downstream_graphs = []
downstream_pos = []
begin=len(dates)-2
downstream_graphs=[sub_nx_graphs[begin]]
downstream_poss=[poss_aligned[begin]]
for i in range (begin-1,0,-1):
    print("i=",i)
    new_graphs,new_poss = second_identification(sub_nx_graphs[i],downstream_graphs[0],poss_aligned[i],downstream_poss[0],50,downstream_graphs[1:],downstream_poss[1:],tolerance=30)
    downstream_graphs=new_graphs
    downstream_poss=new_poss

i= 3
number removed first phase 19
number removed second phase 36
116 0 [16887  8355]
116 1 [16893  8366]
i= 2
number removed first phase 13
number removed second phase 27
121 0 [17317  9088]
121 1 [17312  9091]
121 2 [17311  9107]
i= 1
number removed first phase 12
number removed second phase 32
119 0 [17318  9090]
119 1 [17317  9088]
119 2 [17312  9091]
119 3 [17311  9107]


In [65]:
degree3_nodes = [[node for node in nx_graph.nodes if nx_graph.degree(node)>=3] for nx_graph in downstream_graphs]

In [69]:
t=0
tp1=1
plot_t_tp1(degree3_nodes[t],degree3_nodes[tp1],downstream_poss[t],downstream_poss[tp1],sub_skeleton[t+1].todense(),sub_skeleton[tp1+1].todense(),shift=(xbegin,ybegin))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
downstream_graphs = []
downstream_pos = []
begin=len(dates)-2
downstream_graphs=[nx_graph_pruned[begin]]
downstream_poss=[poss_aligned[begin]]
for i in range (begin-1,0,-1):
    print("i=",i)
    new_graphs,new_poss = second_identification(nx_graph_pruned[i],downstream_graphs[0],poss_aligned[i],downstream_poss[0],50,downstream_graphs[1:],downstream_poss[1:],tolerance=30)
    downstream_graphs=new_graphs
    downstream_poss=new_poss

i= 3
number removed first phase 94
number removed second phase 360
2347 0 [19953  1202]
2347 1 [19967  1222]
i= 2
number removed first phase 143
number removed second phase 621
2093 0 [19962  1191]
2093 1 [19953  1202]
2093 2 [19967  1222]
i= 1
number removed first phase 170
number removed second phase 710
1948 0 [19968  1200]
1948 1 [19962  1191]
1948 2 [19953  1202]
1948 3 [19967  1222]


In [16]:
nx_graph_pruned[1:begin+1]=downstream_graphs
poss_aligned[1:begin+1]=downstream_poss

In [79]:
len(nx_graph_pruned)

6

In [75]:
len(skeleton_docs_aligned)

6

In [17]:
skeleton_docs_cleaned= [generate_skeleton(nx_graph,skeleton_docs_aligned[i].shape) for i,nx_graph in enumerate(nx_graph_pruned)]

In [18]:
for i, date in enumerate(dates):
    sparse.save_npz(f'Data/skeleton_{date}_{plate}_full_clean',sparse.csc_matrix(skeleton_docs_cleaned[i]))

In [20]:
tabs_labeled = [from_nx_to_tab(nx_graph,poss_aligned[i]) for i, nx_graph in enumerate(nx_graph_pruned)]

In [ ]:
def from_nx_to_tab(nx_graph,pos):
    column_names = ["origin_label","end_label","origin_pos", "end_pos", "pixel_list"]
    tab = pd.DataFrame(columns=column_names)
    for edge in nx_graph.edges:
        origin_label=edge[0]
        end_label=edge[1]
        origin_pos=pos[origin_label]
        end_pos = pos[end_label]
        pixel_list=orient(nx_graph.get_edge_data(*edge)['pixel_list'],origin_pos)
        new_line=pd.DataFrame({"origin_label":[origin_label],"end_label" : [end_label], "origin_pos":[origin_pos], "end_pos" : [end_pos], "pixel_list" : [pixel_list]})
        tab=tab.append(new_line,ignore_index=True)
    return(tab)

In [21]:
tabs = [from_sparse_to_graph(skeleton) for skeleton in skeleton_docs_cleaned]

In [22]:
for i, date in enumerate(dates):
    tabs_labeled[i].to_csv(get_path(date,plate,True,extension='_full_labeled.csv'))
    tabs[i].to_csv(get_path(date,plate,True,extension='_full.csv'))
    

FileNotFoundError: [Errno 2] No such file or directory: '//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE/20200701_1957_Plate13/Analysis/Skeleton_full_labeled.csv'

In [23]:
for i, date in enumerate(dates):
    tabs_labeled[i].to_csv(f'Data/graph_{date}_{plate}_full_labeled.csv')
    tabs[i].to_csv(f'Data/graph_{date}_{plate}_full.csv')